<a href="https://colab.research.google.com/github/FabioSicca/Organizacion-de-datos/blob/main/Spark4_TP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

24) Dado un tamaño de vocabulario parametrizable y una lista de stopwords también parametrizable implemente tf-IDF para los textos de los contenidos de forma distribuida. Debe obtener un vector por cada texto (⭐⭐⭐).

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u362-ga-0ubuntu1~20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
type(sc)

pyspark.context.SparkContext

In [ ]:
archivo = spark.read.csv("/content/drive/MyDrive/Wikipedia dataset/contents_text_sample.csv", header=True, inferSchema=True, multiLine=True, escape='"')

In [ ]:
archivo.show(10, truncate=300) #El id coincide con el id de la página de contents.csv

+---+---------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| id|          title|                                                                                                                                                                                                                                                                                                        text|
+---+---------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| 50|  Anacardiaceae|{{Ficha de

In [ ]:
rdd_texto = archivo.rdd

Se puede ver que el texto está escrito con un formato determinado.

In [ ]:
rdd_texto.first()

Row(id=50, title='Anacardiaceae', text="{{Ficha de taxón\n| name = Anacardiáceas\n| image = Gui1 cashewfruit2.jpg\n| image_caption = ''[[Anacardium occidentale]]''\n| regnum = [[Plantae]]\n| divisio  = [[Magnoliophyta]]\n| classis  = [[Magnoliopsida]]\n| ordo = [[Sapindales]]\n| familia = '''Anacardiaceae'''\n| familia_authority = [[Robert Brown|R.Br.]]\n| subdivision_ranks = Géneros\n| subdivision = \nVer texto\n}}\n'''Anacardiaceae''' es una [[familia (biología)|familia]] de [[Plantae|plantas]] esencialmente [[árbol|arbóreas]] y [[arbusto|arbustivas]] perteneciente al [[Orden (biología)|orden]] [[Sapindales]]. La constituyen 77 [[género (biología)|géneros]] con unas 700 especies aceptadas, de las casi 3000 descritas, propias de países tropicales, cálidos y templados.<ref>[http://www.theplantlist.org/1.1/browse/A/Anacardiaceae/ Anacardiaceae en The Plant List, vers. 1.1, 2013]</ref>\n\n== Descripción ==\nSon árboles, arbustos, raramente subarbustos o [[trepadora]]s, frecuentemente con

In [ ]:
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#Defino las stopwords
vocabulary = set()
stopwords_esp = list(set(stopwords.words("spanish")))
mas_palabras = ["cada", "va", "van", "[", "]", ",", "¿", "?", "(", ")", "«", "|", "{","}","=", "|", ".", "»", ":", "", "''",'``', "=", "==", "<", ">", ";", "*"]
stopwords_esp.extend(mas_palabras)

In [ ]:
#Tokenizo las palabras de cada texto
rdd_texto_r = sc.parallelize(rdd_texto.take(5000)) #Tomo una muestra del rdd ya que este es muy extenso y Colab falla
rdd_texto_filtrado = rdd_texto_r.filter(lambda x: len(x.text) > 0 or x.text is not None)
rdd_palabras = rdd_texto_r.map(lambda x: (x.id, word_tokenize(x.text.lower()))) #registro=(id, lista_palabras)
rdd_palabras.count()
#rdd_palabras.take(5)

5000

In [ ]:
#Elimino las palabras que se encuentran entre las stopwords
rdd_palabras_sin_stopwords = rdd_palabras.map(lambda x: (x[0], [palabra for palabra in x[1] if palabra not in stopwords_esp])) #registro=(id, lista_palabras_sin_stopwords)
rdd_palabras_sin_stopwords.cache()
#rdd_palabras_sin_stopwords.take(5)

PythonRDD[194] at RDD at PythonRDD.scala:53

In [ ]:
#Cuento la cantidad de palabras para poder obtener TF(Term frequency) donde TF = (frec palabra por doc / cant palabras por doc)
rdd_palabras_separadas = rdd_palabras_sin_stopwords.flatMap(lambda x: [((x[0],palabra), 1) for palabra in x[1]])  #registro=((id,palabra), 1)

rdd_frec_palabra_por_doc = rdd_palabras_separadas.reduceByKey(lambda a,b: a+b) #registro=((id,palabra), frec)
rdd_aux = rdd_frec_palabra_por_doc.map(lambda x: ((x[0][0], (x[0][1], x[1])))) #registro=(id, (palabra,frec))
rdd_cant_palabras_por_doc = rdd_palabras_sin_stopwords.map(lambda x: (x[0], len(x[1]))) #registro=(id, cantidad_palabras_del_registro)

rdd_TF = rdd_aux.join(rdd_cant_palabras_por_doc).map(lambda x: (x[0], (x[1][0][0],x[1][0][1]/x[1][1]))) #registro=(id, (palabra,TF))

print(rdd_frec_palabra_por_doc.take(3))
print(rdd_TF.take(3))

[((50, 'name'), 3), ((50, 'anacardiáceas'), 1), ((50, 'image'), 1)]
[(160, ('distinguir|analfabeto|alfabetización', 0.0005271481286241434)), (160, ('archivo', 0.0015814443858724301)), (160, ('phönizisch-5sprachen.svg|thumb|el', 0.0005271481286241434))]


In [ ]:
import math as m

#Calculo el IDF(Inverse Document Frequency) IDF=log(cant_docs/cant_docs_que_contienen_la_palabra)
cant_docs = rdd_texto_r.count()

rdd_cant_docs_con_palabra = rdd_palabras_separadas.distinct()\
.map(lambda x: (x[0][1], 1)).reduceByKey(lambda a,b: a+b) #registro=(palabra, cant_docs_donde_aparece)

rdd_IDF_por_palabra = rdd_cant_docs_con_palabra.map(lambda x: (x[0], m.log(cant_docs+1/x[1])))

#rdd1.take(10)
rdd_IDF_por_palabra.take(20)

[('vuelta', 8.517193862556951),
 ('veracidad', 8.517200883694343),
 ('supremacía', 8.51720008794418),
 ('metal', 8.517194542766676),
 ('morden', 8.517393171418904),
 ('alar', 8.517226524194028),
 ("'18", 8.51720430246562),
 ('consecuencias', 8.517194580304162),
 ('marinis', 8.517393171418904),
 ('business', 8.51719676283843),
 ('combustibles', 8.51720059879621),
 ('alkyne', 8.517393171418904),
 ('boxeo', 8.517198905685625),
 ('discurso', 8.51719464069055),
 ('3atext', 8.517201191384238),
 ('exposure', 8.517226524194028),
 ('year', 8.517195191414238),
 ('bloqueando', 8.517221762436654),
 ('reducciones', 8.517218191103742),
 ('georges', 8.517196700182012)]

In [ ]:
rdd_aux_1 = rdd_TF.map(lambda x: (x[1][0], (x[0],x[1][1])) ) #registro = (palabra, (id,TF))
rdd_aux_2 = rdd_aux_1.join(rdd_IDF_por_palabra) #registro= (palabra, ((id,TF),IDF))

rdd_TF_IDF = rdd_aux_2.map(lambda x: (x[1][0][0], (x[0], x[1][0][1]*x[1][1]))).groupByKey().mapValues(list) #registro=(id, lista_de(palabra,TF*IDF))

#rdd_aux_2.take(3)
rdd_TF_IDF.take(3)

[(972,
  [('alfabeto', 0.00034779680547144257),
   ('dio', 0.0006955934119574591),
   ('izquierda', 0.0006955934299656035),
   ('lengua', 0.011825088264019165),
   ('conjunto', 0.005216950614415995),
   ('orden', 0.003130170337919436),
   ('s', 0.003825763672673827),
   ('primeras', 0.0003477967088463713),
   ('vez', 0.003477966981595394),
   ('nombre', 0.006608137235534324),
   ('fin', 0.0013911868125534455),
   ('misma', 0.0024345769156653464),
   ('ejemplo', 0.0006955934030108147),
   ('símbolos', 0.00034779673435767617),
   ('costa', 0.0017389835536530762),
   ('votiva', 0.0003477977128957386),
   ('signos', 0.0006955935696958575),
   ('representa', 0.0003477967144786082),
   ('primeros', 0.002434576915399035),
   ('cadena', 0.00034779673370998734),
   ('constante', 0.0010433901826511181),
   ('primer', 0.005216950475889514),
   ('toda', 0.005564747233596554),
   ('cuyo', 0.001738983531975586),
   ('nbsp', 0.07964544429690242),
   ('22', 0.0027823736307512883),
   ('lenguas', 0.009

De esta forma para cada articulo de Wikipedia se obtendrá un vector que calcula el TF-IDF en forma de registro con la siguiente estructura:
registro=(id_articulo_wiki, lista_de(palabra, TF-IDF))

#Código en limpio

In [ ]:
rdd_texto = archivo.rdd

#Defino las stopwords
vocabulary = set()
stopwords_esp = list(set(stopwords.words("spanish")))
mas_palabras = ["cada", "va", "van", "[", "]", ",", "¿", "?", "(", ")", "«", "|", "{","}","=", "|", ".", "»", ":", "", "''",'``', "=", "==", "<", ">", ";", "*"]
stopwords_esp.extend(mas_palabras)

#Tokenizo las palabras de cada texto
rdd_texto_r = sc.parallelize(rdd_texto.take(5000))   #Tomo una muestra del rdd ya que este es muy extenso y Colab falla
rdd_texto_filtrado = rdd_texto_r.filter(lambda x: len(x.text) > 0 or x.text is not None)
rdd_palabras = rdd_texto_r.map(lambda x: (x.id, word_tokenize(x.text.lower()))) #registro=(id, lista_palabras)

#Elimino las palabras que se encuentran entre las stopwords
rdd_palabras_sin_stopwords = rdd_palabras.map(lambda x: (x[0], [palabra for palabra in x[1] if palabra not in stopwords_esp])) #registro=(id, lista_palabras_sin_stopwords)
rdd_palabras_sin_stopwords.cache()

rdd_palabras_separadas = rdd_palabras_sin_stopwords.flatMap(lambda x: [((x[0],palabra), 1) for palabra in x[1]]) #registro=((id,palabra), 1)

#Cuento la cantidad de palabras para poder obtener TF(Term frequency) donde TF = (frec palabra por doc / cant palabras por doc)
rdd_frec_palabra_por_doc = rdd_palabras_separadas.reduceByKey(lambda a,b: a+b) #registro=((id,palabra), frec)
rdd_aux = rdd_frec_palabra_por_doc.map(lambda x: ((x[0][0], (x[0][1], x[1])))) #registro=(id, (palabra,frec))
rdd_cant_palabras_por_doc = rdd_palabras_sin_stopwords.map(lambda x: (x[0], len(x[1]))) #registro=(id, cantidad_palabras_del_registro)

rdd_TF = rdd_aux.join(rdd_cant_palabras_por_doc).map(lambda x: (x[0], (x[1][0][0],x[1][0][1]/x[1][1]))) #registro=(id, (palabra,TF))

#Calculo el IDF(Inverse Document Frequency) IDF=log(cant_docs/cant_docs_que_contienen_la_palabra)
cant_docs = rdd_texto_r.count()

rdd_cant_docs_con_palabra = rdd_palabras_separadas.distinct()\
.map(lambda x: (x[0][1], 1)).reduceByKey(lambda a,b: a+b) #registro=(palabra, cant_docs_donde_aparece)

rdd_IDF_por_palabra = rdd_cant_docs_con_palabra.map(lambda x: (x[0], m.log(cant_docs+1/x[1])))


rdd_aux_1 = rdd_TF.map(lambda x: (x[1][0], (x[0],x[1][1])) ) #registro = (palabra, (id,TF))
rdd_aux_2 = rdd_aux_1.join(rdd_IDF_por_palabra) #registro= (palabra, ((id,TF),IDF))

rdd_TF_IDF = rdd_aux_2.map(lambda x: (x[1][0][0], (x[0], x[1][0][1]*x[1][1]))).groupByKey().mapValues(list) #registro=(id, lista_de(palabra,TF*IDF))
rdd_TF_IDF.take(3) #.collect() si quiero obtener todos los resultados.

[(972,
  [('alfabeto', 0.00034779680547144257),
   ('dio', 0.0006955934119574591),
   ('izquierda', 0.0006955934299656035),
   ('lengua', 0.011825088264019165),
   ('conjunto', 0.005216950614415995),
   ('orden', 0.003130170337919436),
   ('s', 0.003825763672673827),
   ('primeras', 0.0003477967088463713),
   ('vez', 0.003477966981595394),
   ('nombre', 0.006608137235534324),
   ('fin', 0.0013911868125534455),
   ('misma', 0.0024345769156653464),
   ('ejemplo', 0.0006955934030108147),
   ('símbolos', 0.00034779673435767617),
   ('costa', 0.0017389835536530762),
   ('votiva', 0.0003477977128957386),
   ('signos', 0.0006955935696958575),
   ('representa', 0.0003477967144786082),
   ('primeros', 0.002434576915399035),
   ('cadena', 0.00034779673370998734),
   ('constante', 0.0010433901826511181),
   ('primer', 0.005216950475889514),
   ('toda', 0.005564747233596554),
   ('cuyo', 0.001738983531975586),
   ('nbsp', 0.07964544429690242),
   ('22', 0.0027823736307512883),
   ('lenguas', 0.009